In [ ]:
from pathlib import Path
import pandas as pd
import re
from numpy import nan

In [ ]:
iso_table = pd.read_csv(
    "https://raw.githubusercontent.com/sepal-contrib/sepal_mgci/master/component/parameter/m49_countries.csv",
    sep=";",
)
belt_table = pd.read_csv("data/sources/5_belts_translation.csv")
mgci_columns = [
    "SeriesID",
    "SeriesDescription",
    "GeoAreaCode",
    "GeoAreaName",
    "TimePeriod",
    "Value",
    "Time_Detail",
    "Source",
    "FootNote",
    "Nature",
    "Units",
    "Reporting Type",
    "Observation Status",
    "Bioclimatic Belt",
    "Land Type",
    "ISOalpha3",
    "Type",
    "SeriesCode",
]
lcluc_matrix = pd.read_csv("data/sources/cci_map_matrix.csv")
lcluc_matrix = lcluc_matrix[["target_code", "green"]]
lcluc_matrix.drop_duplicates(keep="first", inplace=True)
lcluc_matrix = lcluc_matrix.reset_index(drop=True)

lcluc_table = pd.read_csv("data/sources/cci_map_matrix.csv")
lcluc_table = (
    lcluc_table.groupby(["target_code", "target_desc"])
    .sum()
    .reset_index()[["target_code", "target_desc"]]
)

In [ ]:
def get_belt_desc(row):
    """return bioclimatic belt description"""
    desc = belt_table[belt_table.code == row["belt_class"]]["desc"]

    return desc.values[0] if len(desc) else "Total"


def get_lc_desc(row):
    """return landcover description"""
    desc = lcluc_table[lcluc_table.target_code == row["lc_class"]]["target_desc"]

    return desc.values[0] if len(desc) else "All"

In [ ]:
def get_mgci_report(df, iso_code):
    """
    Get the MGCI report table for the given iso_code and year

    df (DataFrame): grouped dataframe from raw data
    iso_code (str): country iso code
    """

    # subset dataframe to the one we are working on
    df_ = df[df.iso_code == iso_code].copy(deep=True)
    # Check lenght, if zero (0), there are not belts in the country.
    if not len(df_):
        return pd.DataFrame([[iso_code, None, None, None]])

    # "iso_code", "belt_class", target, "sum"
    belt_area = (
        df_.groupby(["iso_code", "belt_class"])
        .sum()
        .reset_index()[["iso_code", "belt_class", "sum"]]
    )
    belt_area["lc_class"] = "All"

    df_ = pd.concat([df_, belt_area])

    by_lc = (
        df_.groupby(["iso_code", "lc_class"])
        .sum()
        .reset_index()[["iso_code", "lc_class", "sum"]]
    )
    by_lc["belt_class"] = "Total"

    return pd.concat([df_, by_lc])

In [ ]:
i = 0


def create_report(df, iso_code, target_year):
    global i
    i += 1
    try:
        country_df = get_mgci_report(df, iso_code)
        report_df = country_df
        report_df["SeriesID"] = i
        report_df["Value"] = report_df["sum"]
        report_df["SeriesDescription"] = "Mountain Green Cover Index"
        report_df["GeoAreaName"] = report_df.iso_code
        report_df["GeoAreaCode"] = iso_table[
            iso_table.iso31661 == report_df.GeoAreaName.unique()[0]
        ]["m49"].iloc[0]
        report_df["TimePeriod"] = target_year
        report_df["Time_Detail"] = target_year
        report_df[
            "Source"
        ] = "Food and Agriculture Organisation of United Nations (FAO)"
        report_df["FootNote"] = "FAO estimate"
        report_df["Nature"] = "G"
        report_df["Units"] = "SQKM_PA"
        report_df["Reporting Type"] = "G"
        report_df["Observation Status"] = "A"
        report_df["Bioclimatic Belt"] = report_df.apply(get_belt_desc, axis=1)
        report_df["ISOalpha3"] = nan
        report_df["Land Type"] = report_df.apply(get_lc_desc, axis=1)
        report_df["Type"] = "Region"
        report_df["SeriesCode"] = "ER_MTN_GRNCVI"
        return report_df

    except Exception as e:
        print(iso_code)
        raise e

## Loop over years

In [ ]:
class Progress:
    def __init__(self):
        self.processed_years = []

In [ ]:
def report_by_dataset(csv_path):
    """Create and save the subindicator for each of the dates in the given csv path

    csv_path: Has to be created using the reduce_results notebook. It has the formatted data.

    """

    raw_df = pd.read_csv(csv_path)

    # Extract years from the CSV file
    years = [
        int(x) for x in re.search(r"\d{4}_\d{4}", str(csv_path)).group().split("_")
    ]

    def report_date(year):

        target_year = year

        target = "from_class" if years.index(target_year) == 0 else "to_class"
        print(f"Processing {target_year}, {target} for {csv_path}")

        df = (
            raw_df.groupby(
                by=[
                    "iso_code",
                    "belt_class",
                    target,
                ]
            )
            .sum()
            .reset_index()
        )
        df = df[["iso_code", "belt_class", target, "sum"]]
        df.rename(columns={target: "lc_class"}, inplace=True)

        countries_df = [
            create_report(df, iso_code, target_year)
            for iso_code in df.iso_code.unique()
        ]

        final = pd.concat(countries_df)[mgci_columns]
        return final

    # To this for the two years
    return pd.concat([report_date(year) for year in years])

In [ ]:
# # Test with one country
# raw_df = pd.read_csv("data/global_group_1_2_3/global_grouped_data_2015_2018.csv")
# df = raw_df.groupby(by=["iso_code", "belt_class", "from_class", ]).sum().reset_index()[["iso_code", "belt_class", "from_class", "sum"]]
# df.rename(columns={"from_class":"lc_class"}, inplace=True)

# create_report(df, "COL", 2020)

In [ ]:
# Now let's do this for each of the datasets

In [ ]:
progress = Progress()
reports_by_year = [
    report_by_dataset(csv_path)
    for csv_path in Path("data/global_group_1_2_3").glob("global*.csv")
]

In [ ]:
reports_by_year_df = pd.concat(reports_by_year)
reports_by_year_df = reports_by_year_df.sort_values(
    ["GeoAreaName", "TimePeriod", "Bioclimatic Belt"]
)

In [ ]:
out_path = Path("data/reporting/subA_landtype/")
out_path.mkdir(exist_ok=True, parents=True)
reports_by_year_df.to_csv(out_path / f"SubIndA_LandType.csv")

In [ ]:
path = Path("data/reporting/subA_landtype/")
pd.concat([pd.read_csv(csv) for csv in path.glob("[!.]*.csv")]).to_csv(
    path / "SubIndA_LandType.csv"
)

## Test with a single country


In [ ]:
csv_path = Path("data/global_group_1_2_3/global_grouped_data_2000_2015.csv")

raw_df = pd.read_csv(csv_path)

# Extract years from the CSV file
years = [int(x) for x in re.search(r"\d{4}_\d{4}", str(csv_path)).group().split("_")]

target_year = 2000

target = "from_class" if years.index(target_year) == 0 else "to_class"

df = (
    raw_df.groupby(
        by=[
            "iso_code",
            "belt_class",
            target,
        ]
    )
    .sum()
    .reset_index()
)
df = df[["iso_code", "belt_class", target, "sum"]]
df.rename(columns={target: "lc_class"}, inplace=True)

In [ ]:
test_df = get_mgci_report(df, "AFG")

In [ ]:
test_df

In [ ]:
by_lc = (
    test_df.groupby(["iso_code", "lc_class"])
    .sum()
    .reset_index()[["iso_code", "lc_class", "sum"]]
)
by_lc["belt_class"] = "Total"
by_lc

In [ ]:
pd.concat([test_df, by_lc])